## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-04-05-13-59-42.csv',dtype='str')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['March']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    1155
Name: PRICE, dtype: int64
-------
False    1139
True       16
Name: $/SQUARE FEET, dtype: int64
-------
False    1155
Name: YEAR BUILT, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
618,PAST SALE,March-13-2023,Condo/Co-op,533 NE 3rd Ave #341,Fort Lauderdale,FL,33301,295,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/533-...,Beaches MLS,F10362350,N,Y,26.128875,-80.141375
832,PAST SALE,March-13-2023,Condo/Co-op,1044 SW 147 Ave #1044,Pembroke Pines,FL,33027,355,2,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1044-...,MARMLS,A11297383,N,Y,25.999574,-80.342651
532,PAST SALE,March-3-2023,Condo/Co-op,3080 Holiday Springs Blvd #207,Margate,FL,33063,52100,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3080-Holiday...,Beaches MLS,F10364092,N,Y,26.262281,-80.233221
34,PAST SALE,March-29-2023,Condo/Co-op,3051 NW 46th Ave #304,Lauderdale Lakes,FL,33313,60000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/305...,Beaches MLS,F10368502,N,Y,26.164006,-80.210737
38,PAST SALE,March-24-2023,Condo/Co-op,5680 SW 3rd Pl #103,Margate,FL,33068,68000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/5680-SW-3rd-...,MARMLS,A11345051,N,Y,26.226623,-80.203056
896,PAST SALE,March-15-2023,Condo/Co-op,2551 NW 41st Ave #108,Lauderhill,FL,33313,77000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2551-NW-4...,Beaches MLS,RX-10850760,N,Y,26.158183,-80.204546
855,PAST SALE,March-31-2023,Condo/Co-op,4880 NW 22nd St #204,Lauderhill,FL,33313,78000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4880-NW-2...,Beaches MLS,F10361312,N,Y,26.153611,-80.216402
143,PAST SALE,March-15-2023,Condo/Co-op,4310 NW 12th Ct #308,Lauderhill,FL,33313,80000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4310-NW-1...,Beaches MLS,F10363049,N,Y,26.141413,-80.207154
1083,PAST SALE,March-2-2023,Condo/Co-op,6070 NW 64th Ave #206,Tamarac,FL,33319,80000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/6070-NW-64th...,Beaches MLS,RX-10832302,N,Y,26.199981,-80.234527
41,PAST SALE,March-17-2023,Condo/Co-op,3071 E Sunrise Lakes Dr #103,Sunrise,FL,33322,80100,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/3071-E-Sunri...,MARMLS,A11163383,N,Y,26.162376,-80.251037


In [11]:
print(df_filtered['URL'].iloc[832])

https://www.redfin.com/FL/Pembroke-Pines/1044-SW-147th-Ave-33027/unit-1044/home/145542501


In [12]:
# Correct the prices, if needed
df_filtered.at[618,'PRICE']=(295000)
df_filtered.at[832,'PRICE']=(355000)

In [13]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [177]:
# # Corrections, if needed
# df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
# df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [14]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
532,3080 Holiday Springs Blvd #207,Margate,72.0
34,3051 NW 46th Ave #304,Lauderdale Lakes,78.0
20,4157 NW 90th Ave #104,Coral Springs,84.0
43,3910 Inverrary Blvd Unit 701-B,Lauderhill,85.0
712,4751 NW 21st St #206,Lauderhill,88.0
351,2800 Somerset Dr Unit 416J,Lauderdale Lakes,91.0
1073,3001 NW 46th Ave #402,Lauderdale Lakes,93.0
370,2061 NW 47th Ter #409,Lauderhill,94.0
41,3071 E Sunrise Lakes Dr #103,Sunrise,94.0
917,6195 Rock Island Rd #409,Tamarac,94.0


In [92]:
# print(df_filtered.URL.iloc[582])

In [93]:
# Corrections, if needed
#
#
#

In [94]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [15]:
df_Current = df_filtered.loc[df_filtered['SOLD DATE'].str.contains('March')]

In [16]:
### Create a price column formatted as currency ###
df_Current['PRICE_AS_CURRENCY'] = df_Current['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_Current['YEAR BUILT'] = df_Current['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_Current['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_Current['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [17]:
df_Current = df_Current.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_Current['COLOR'] = ''
### Create RANK column ###
df_Current['RANK'] = 0
### Insert RANK values ###
df_Current['RANK'] = range(1, len(df_Current) + 1)
# use numpy to assign values to the 'COLOR' column
df_Current['COLOR'] = np.where(df_Current['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [18]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [19]:
### Drop the columns ###
df_Current = df_Current.drop(columns=columns_drop)

In [20]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [24]:
### Create map container ###
m = folium.Map(location=df_Current[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"March 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_Current.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [25]:
m.save('index.html')

## Summary Info

In [26]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [27]:
df_Current.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'PRICE_SQUARE_FEET_AS_CURRENCY',
       'COLOR', 'RANK'],
      dtype='object')

In [28]:
df_Current['FULL_ADDRESS'] = df_Current['ADDRESS'] + ' ' + df_Current['CITY']

In [29]:
df_Current['YEAR BUILT'] = pd.to_numeric(df['YEAR BUILT'])

In [30]:
print(f"{ME}{BR}{df_Current.loc[df_Current['PRICE'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df_Current.loc[df_Current['PRICE'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
Four Seasons, 525 Ft. Lauderdale Bch Bln #1403 Fort Lauderdale | Price $5,350,000 | $2,286 psf | Year built: 1977.0
Least Expensive
Holiday Springsvillage, 3080 Holiday Springs Blvd #207 Margate | Price $52,100 | $72 psf | Year built: 1969.0
Highest Price Per Square Foot
Four Seasons, 525 Ft. Lauderdale Bch Bln #1403 Fort Lauderdale | Price $5,350,000 | $2,286 psf | Year built: 1977.0
Lowest Price Per Square Foot
Holiday Springsvillage, 3080 Holiday Springs Blvd #207 Margate | Price $52,100 | $72 psf | Year built: 1969.0
Newest
Marlin Towers, 3200 NE 29th St #302 Fort Lauderdale | Price $370,000 | $370.0 psf | Year built: 2022.0
Oldest
Wynmoor Village, 4301 Martinique Cir Unit E3 Coconut Creek | Price $255,000 | $204.0 psf | Year built: 1957.0


## Time on Market Calculator

In [415]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [225]:
from datetime import datetime, timedelta

date1 = datetime(2022, 10, 26) ## List (Earlier) date
date2 = datetime(2023, 3, 8) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

133


## Map URL Snagger

In [194]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [195]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Palm_Beach_condo_sales_month_ending_march_2023


## Get Summary Data

In [196]:
print('SALES INFO')
print(f'Number of sales: {len(df_Current)}')
print('--------')
print(f'Total sale price: ${df_Current["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_Current["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_Current["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_Current["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_Current["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_Current["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_Current["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_Current["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_Current["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_Current["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 942
--------
Total sale price: $460,616,290
--------
Median sale price: $265,000
--------
Max sale price: $10,650,000
--------
Min sale price: $54,956
------------------------------------------------
PSF INFO
Max price per square foot: $3,122
--------
Min price per square foot: $60
--------
Median price per square foot: $232
------------------------------------------------
CONDO AGES
Newest building: 2021.0
----------
Oldest building: 1925.0
----------
Average building age: 1982.6240409207162


In [227]:
df_Current.head(60)

,SOLD DATE,ADDRESS,CITY,PRICE,BEDS,BATHS,LOCATION,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,URL,LATITUDE,LONGITUDE,PRICE_AS_CURRENCY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS
0,March-14-2023,1500 Beach Rd #601,Tequesta,10650000,3,4.5,Seaglass Condominium,1988.0,NaN,2964.0,https://www.redfin.com/FL/Jupiter/1500-S-Beach...,26.961813,-80.079386,"$10,650,000","$2,964",orange,1,1500 Beach Rd #601 Tequesta
1,March-16-2023,1000 S Ocean Blvd #505,Boca Raton,9225000,3,3.5,One Thousand Ocean,1980.0,NaN,2244.0,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,26.336983,-80.071216,"$9,225,000","$2,244",orange,2,1000 S Ocean Blvd #505 Boca Raton
2,March-31-2023,315 S Lake Dr Ph C,Palm Beach,8000000,3,3.0,Southlake Condominium,1974.0,NaN,2957.0,https://www.redfin.com/FL/Palm-Beach/315-S-Lak...,26.705174,-80.043319,"$8,000,000","$2,957",orange,3,315 S Lake Dr Ph C Palm Beach
3,March-29-2023,360 S Ocean Blvd Unit 4c,Palm Beach,7650000,3,3.5,Dunster House Condo,1992.0,NaN,3122.0,https://www.redfin.com/FL/Palm-Beach/360-S-Oce...,26.702626,-80.033669,"$7,650,000","$3,122",orange,4,360 S Ocean Blvd Unit 4c Palm Beach
4,March-28-2023,2 N Breakers Row Unit S43,Palm Beach,6700000,2,2.5,Two North Breakers Row Condo,1979.0,NaN,2443.0,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,26.717657,-80.034595,"$6,700,000","$2,443",orange,5,2 N Breakers Row Unit S43 Palm Beach
5,March-9-2023,100 Worth Ave Unit 713 & Ph13,Palm Beach,5500000,2,4.0,Winthrop House Condo,NaN,NaN,1698.0,https://www.redfin.com/FL/Palm-Beach/100-Worth...,26.700385,-80.033799,"$5,500,000","$1,698",orange,6,100 Worth Ave Unit 713 & Ph13 Palm Beach
6,March-24-2023,400 N Flagler Dr Ph -C6,West Palm Beach,4560225,3,2.5,Waterview Towers Condo,1990.0,NaN,1572.0,https://www.redfin.com/FL/West-Palm-Beach/400-...,26.716407,-80.050131,"$4,560,225","$1,572",orange,7,400 N Flagler Dr Ph -C6 West Palm Beach
7,March-1-2023,429 Australian Ave #100,Palm Beach,4525400,3,3.0,Australian OF Palm Beach Condo,2002.0,NaN,2178.0,https://www.redfin.com/FL/Palm-Beach/429-Austr...,26.703642,-80.042271,"$4,525,400","$2,178",orange,8,429 Australian Ave #100 Palm Beach
8,March-6-2023,3140 S Ocean Blvd Unit 503n,Palm Beach,4250000,3,3.0,Carlton Place Condo,1987.0,NaN,1577.0,https://www.redfin.com/FL/Palm-Beach/3140-S-Oc...,26.606269,-80.037337,"$4,250,000","$1,577",orange,9,3140 S Ocean Blvd Unit 503n Palm Beach
9,March-27-2023,3120 S Ocean Blvd Unit 3-502,Palm Beach,3375000,2,2.5,Thirty One Twenty Condo,1981.0,NaN,1352.0,https://www.redfin.com/FL/Palm-Beach/3120-S-Oc...,26.607497,-80.037294,"$3,375,000","$1,352",orange,10,3120 S Ocean Blvd Unit 3-502 Palm Beach
